In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
import statsmodels.api as sm
from scipy import stats
import os
from tqdm import tqdm
import multiprocessing as mp
from functools import partial

In [ ]:
def perform_two_way_anova(bin_data, factor1_name, factor2_name):
    """
    Perform two-way ANOVA on a single bin's data
    
    Parameters:
    -----------
    bin_data : pandas.DataFrame
        DataFrame with columns for 'measurement', factor1_name, and factor2_name
    factor1_name : str
        Name of the first factor
    factor2_name : str
        Name of the second factor
        
    Returns:
    --------
    dict
        Dictionary with p-values and F-values for each factor and interaction
    """
    try:
        # Create the formula for the model
        formula = f"measurement ~ C({factor1_name}) + C({factor2_name}) + C({factor1_name}):C({factor2_name})"
        
        # Fit the model (using Type III SS)
        model = ols(formula, data=bin_data).fit()
        
        # Get ANOVA table
        anova_table = sm.stats.anova_lm(model, typ=3)
        
        # Extract results
        results = {
            'factor1_pvalue': anova_table.loc[f'C({factor1_name})', 'PR(>F)'],
            'factor2_pvalue': anova_table.loc[f'C({factor2_name})', 'PR(>F)'],
            'interaction_pvalue': anova_table.loc[f'C({factor1_name}):C({factor2_name})', 'PR(>F)'],
            'factor1_Fvalue': anova_table.loc[f'C({factor1_name})', 'F'],
            'factor2_Fvalue': anova_table.loc[f'C({factor2_name})', 'F'],
            'interaction_Fvalue': anova_table.loc[f'C({factor1_name}):C({factor2_name})', 'F']
        }
        
        return results
    
    except Exception as e:
        # Return NaN values if error occurs
        return {
            'factor1_pvalue': np.nan,
            'factor2_pvalue': np.nan,
            'interaction_pvalue': np.nan,
            'factor1_Fvalue': np.nan,
            'factor2_Fvalue': np.nan,
            'interaction_Fvalue': np.nan
        }


In [ ]:
def process_bin(bin_idx, bin_id, bin_data, metadata, factor1_name, factor2_name):
    """Process a single bin - for parallel processing"""
    # Extract measurements for this bin
    bin_values = bin_data.iloc[bin_idx, 1:].values
    
    # Create DataFrame with measurements and factors
    df = pd.DataFrame({
        'measurement': bin_values,
        factor1_name: metadata[factor1_name].values,
        factor2_name: metadata[factor2_name].values
    })
    
    # Perform ANOVA
    result = perform_two_way_anova(df, factor1_name, factor2_name)
    result['bin_id'] = bin_id
    
    return result

In [ ]:
def analyze_genomic_bins(data_file, metadata_file, factor1_name, factor2_name, output_prefix, n_cores=None):
    """
    Analyze thousands of genomic bins with two-way ANOVA
    
    Parameters:
    -----------
    data_file : str
        Path to CSV file with bin data (first column is bin_id, other columns are samples)
    metadata_file : str
        Path to CSV file with sample metadata (must have sample_id, factor1_name, and factor2_name columns)
    factor1_name : str
        Name of the first factor in metadata
    factor2_name : str
        Name of the second factor in metadata
    output_prefix : str
        Prefix for output files
    n_cores : int, optional
        Number of CPU cores to use for parallel processing. If None, uses all available cores - 1
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with ANOVA results for all bins
    """
    print("Reading data...")
    # Read data
    data = pd.read_csv(data_file)
    metadata = pd.read_csv(metadata_file)
    
    # Get bin IDs
    bin_ids = data.iloc[:, 0].values
    
    # Ensure the column names in data match the sample_ids in metadata
    data_cols = data.columns[1:]
    metadata_samples = metadata['sample_id'].values
    
    # Check that all data columns are in metadata
    missing_cols = set(data_cols) - set(metadata_samples)
    if missing_cols:
        raise ValueError(f"The following samples in data are missing from metadata: {missing_cols}")
        
    # Ensure the order of columns in data matches the order in metadata
    # We'll reorder the metadata to match the data
    sample_order = {sample: i for i, sample in enumerate(data_cols)}
    metadata['data_order'] = metadata['sample_id'].map(sample_order)
    metadata = metadata.sort_values('data_order').reset_index(drop=True)
    metadata = metadata.drop(columns=['data_order'])
    
    # Check if factors exist in metadata
    if factor1_name not in metadata.columns:
        raise ValueError(f"Factor '{factor1_name}' not found in metadata")
    if factor2_name not in metadata.columns:
        raise ValueError(f"Factor '{factor2_name}' not found in metadata")
    
    # Determine number of cores for parallel processing
    if n_cores is None:
        n_cores = max(1, mp.cpu_count() - 1)  # Leave one core free
    
    print(f"Processing {len(bin_ids)} bins using {n_cores} cores...")
    
    # Initialize results list
    results_list = []
    
    # Set up parallel processing
    with mp.Pool(n_cores) as pool:
        # Create a partial function with fixed arguments
        func = partial(process_bin, 
                      bin_data=data, 
                      metadata=metadata, 
                      factor1_name=factor1_name, 
                      factor2_name=factor2_name)
        
        # Process bins in parallel with progress bar
        for result in tqdm(pool.starmap(func, [(i, bin_id) for i, bin_id in enumerate(bin_ids)]), 
                          total=len(bin_ids)):
            results_list.append(result)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results_list)
    
    # Apply multiple testing correction
    for col in ['factor1_pvalue', 'factor2_pvalue', 'interaction_pvalue']:
        # Benjamini-Hochberg FDR correction
        mask = ~np.isnan(results_df[col])
        corrected = np.full(len(results_df), np.nan)
        
        if mask.sum() > 0:
            corrected[mask] = multipletests(results_df.loc[mask, col], method='fdr_bh')[1]
            
        results_df[f'{col}_adj'] = corrected
        
        # Add significance flag (True/False)
        results_df[f'{col.replace("pvalue", "significant")}'] = results_df[f'{col}_adj'] < 0.05
    
    # Write results to file
    results_df.to_csv(f"{output_prefix}_anova_results.csv", index=False)
    
    print(f"Analysis complete. Found:")
    print(f"  - {results_df['factor1_significant'].sum()} bins significant for {factor1_name}")
    print(f"  - {results_df['factor2_significant'].sum()} bins significant for {factor2_name}")
    print(f"  - {results_df['interaction_significant'].sum()} bins significant for interaction")
    
    return results_df

In [ ]:



def visualize_anova_results(results_df, factor1_name, factor2_name, output_prefix):
    """
    Create visualizations of ANOVA results
    
    Parameters:
    -----------
    results_df : pandas.DataFrame
        DataFrame with ANOVA results from analyze_genomic_bins()
    factor1_name : str
        Name of the first factor
    factor2_name : str
        Name of the second factor
    output_prefix : str
        Prefix for output files
        
    Returns:
    --------
    dict
        Dictionary with matplotlib figure objects
    """
    # Create output directory for plots
    os.makedirs(f"{output_prefix}_plots", exist_ok=True)
    
    print("Creating visualizations...")
    figures = {}
    
    # 1. Manhattan-like plot of p-values
    plt.figure(figsize=(12, 6))
    plt.scatter(range(len(results_df)), -np.log10(results_df['factor1_pvalue']), 
               alpha=0.5, s=10, label=factor1_name)
    plt.scatter(range(len(results_df)), -np.log10(results_df['factor2_pvalue']), 
               alpha=0.5, s=10, label=factor2_name)
    plt.scatter(range(len(results_df)), -np.log10(results_df['interaction_pvalue']), 
               alpha=0.5, s=10, label='Interaction')
    
    # Add significance thresholds
    plt.axhline(-np.log10(0.05), linestyle='--', color='red', label='p=0.05')
    plt.axhline(-np.log10(0.05/len(results_df)), linestyle='--', color='blue', label='Bonferroni')
    
    plt.xlabel('Genomic Bin Index')
    plt.ylabel('-log10(p-value)')
    plt.title('Manhattan Plot of Two-way ANOVA p-values')
    plt.legend()
    plt.tight_layout()
    
    plt.savefig(f"{output_prefix}_plots/manhattan_plot.png", dpi=300)
    plt.savefig(f"{output_prefix}_plots/manhattan_plot.pdf")
    figures['manhattan'] = plt.gcf()
    
    # 2. Volcano plots for each factor
    factor_cols = [
        (factor1_name, 'factor1_pvalue', 'factor1_Fvalue', 'factor1_significant'),
        (factor2_name, 'factor2_pvalue', 'factor2_Fvalue', 'factor2_significant'),
        ('Interaction', 'interaction_pvalue', 'interaction_Fvalue', 'interaction_significant')
    ]
    
    for name, pval_col, fval_col, sig_col in factor_cols:
        plt.figure(figsize=(8, 6))
        
        # Create a mask for non-NaN values
        mask = ~np.isnan(results_df[pval_col]) & ~np.isnan(results_df[fval_col])
        
        # Create a scatter plot with color indicating significance
        plt.scatter(
            results_df.loc[mask, fval_col],
            -np.log10(results_df.loc[mask, pval_col]),
            c=results_df.loc[mask, sig_col].map({True: 'red', False: 'black'}),
            alpha=0.5,
            s=15
        )
        
        plt.axhline(-np.log10(0.05), linestyle='--', color='red')
        plt.xlabel('F value (effect size)')
        plt.ylabel('-log10(p-value)')
        plt.title(f'Volcano Plot: {name}')
        plt.tight_layout()
        
        plt.savefig(f"{output_prefix}_plots/volcano_{name.lower().replace(' ', '_')}.png", dpi=300)
        plt.savefig(f"{output_prefix}_plots/volcano_{name.lower().replace(' ', '_')}.pdf")
        figures[f'volcano_{name.lower().replace(" ", "_")}'] = plt.gcf()
    
    # 3. Distribution of p-values
    plt.figure(figsize=(10, 6))
    
    # Create histograms for each factor
    bins = np.linspace(0, 1, 21)  # 20 bins from 0 to 1
    
    for i, (name, col) in enumerate(zip(
        [factor1_name, factor2_name, 'Interaction'],
        ['factor1_pvalue', 'factor2_pvalue', 'interaction_pvalue']
    )):
        # Skip NaN values
        p_values = results_df[col].dropna()
        if len(p_values) > 0:
            plt.hist(p_values, bins=bins, alpha=0.5, label=name)
    
    plt.xlabel('p-value')
    plt.ylabel('Count')
    plt.title('Distribution of p-values')
    plt.legend()
    plt.tight_layout()
    
    plt.savefig(f"{output_prefix}_plots/pvalue_distribution.png", dpi=300)
    plt.savefig(f"{output_prefix}_plots/pvalue_distribution.pdf")
    figures['pvalue_distribution'] = plt.gcf()
    
    # 4. Venn diagram of significant results
    # Create a summary file with counts
    sig_factor1 = results_df['factor1_significant'].sum()
    sig_factor2 = results_df['factor2_significant'].sum()
    sig_interaction = results_df['interaction_significant'].sum()
    
    sig_factor1_and_factor2 = (results_df['factor1_significant'] & 
                             results_df['factor2_significant']).sum()
    sig_factor1_and_interaction = (results_df['factor1_significant'] & 
                                 results_df['interaction_significant']).sum()
    sig_factor2_and_interaction = (results_df['factor2_significant'] & 
                                 results_df['interaction_significant']).sum()
    sig_all = (results_df['factor1_significant'] & 
              results_df['factor2_significant'] & 
              results_df['interaction_significant']).sum()
    
    # Create a DataFrame with the counts
    venn_summary = pd.DataFrame({
        'Category': [
            'Total Bins',
            f'Significant {factor1_name}',
            f'Significant {factor2_name}',
            'Significant Interaction',
            f'Significant {factor1_name} and {factor2_name}',
            f'Significant {factor1_name} and Interaction',
            f'Significant {factor2_name} and Interaction',
            'Significant in all tests'
        ],
        'Count': [
            len(results_df),
            sig_factor1,
            sig_factor2,
            sig_interaction,
            sig_factor1_and_factor2,
            sig_factor1_and_interaction,
            sig_factor2_and_interaction,
            sig_all
        ]
    })
    
    venn_summary.to_csv(f"{output_prefix}_venn_summary.csv", index=False)
    
    return figures



In [ ]:
def analyze_significant_bins(data_file, metadata_file, results_df, factor1_name, factor2_name, 
                          output_prefix, max_bins=10):
    """
    Analyze and visualize significant bins
    
    Parameters:
    -----------
    data_file : str
        Path to CSV file with bin data
    metadata_file : str
        Path to CSV file with sample metadata
    results_df : pandas.DataFrame
        DataFrame with ANOVA results from analyze_genomic_bins()
    factor1_name : str
        Name of the first factor
    factor2_name : str
        Name of the second factor
    output_prefix : str
        Prefix for output files
    max_bins : int, optional
        Maximum number of top bins to visualize for each factor
        
    Returns:
    --------
    dict
        Dictionary with DataFrames of top bins for each factor
    """
    print("Analyzing significant bins...")
    
    # Read data
    data = pd.read_csv(data_file)
    metadata = pd.read_csv(metadata_file)
    
    # Create output directory for bin plots
    os.makedirs(f"{output_prefix}_bin_plots", exist_ok=True)
    
    # Get top significant bins for each factor
    factor_cols = [
        (factor1_name, 'factor1_pvalue', 'factor1_significant'),
        (factor2_name, 'factor2_pvalue', 'factor2_significant'),
        ('Interaction', 'interaction_pvalue', 'interaction_significant')
    ]
    
    top_bins = {}
    
    for name, pval_col, sig_col in factor_cols:
        # Get significant bins
        sig_bins = results_df[results_df[sig_col]].copy()
        
        # Sort by p-value
        sig_bins = sig_bins.sort_values(pval_col)
        
        # Take top bins
        top = sig_bins.head(min(max_bins, len(sig_bins)))
        
        # Save to file
        top.to_csv(f"{output_prefix}_top_{name.lower().replace(' ', '_')}_bins.csv", index=False)
        
        # Store in dictionary
        top_bins[f'top_{name.lower().replace(" ", "_")}'] = top
        
        # Plot each top bin
        for i, (_, row) in enumerate(top.iterrows()):
            plot_single_bin(data, metadata, row['bin_id'], row, factor1_name, factor2_name, 
                           f"{output_prefix}_bin_plots/top_{name.lower().replace(' ', '_')}_{i+1}_{row['bin_id']}")
    
    return top_bins



In [ ]:
def plot_single_bin(data, metadata, bin_id, stats_row, factor1_name, factor2_name, output_file_prefix):
    """
    Plot a single bin's data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with bin data
    metadata : pandas.DataFrame
        DataFrame with sample metadata
    bin_id : str or int
        ID of the bin to plot
    stats_row : pandas.Series
        Row from results DataFrame with statistics for this bin
    factor1_name : str
        Name of the first factor
    factor2_name : str
        Name of the second factor
    output_file_prefix : str
        Prefix for output file
    """
    # Find the row in the original data corresponding to this bin
    bin_row = data[data.iloc[:, 0] == bin_id]
    
    if bin_row.empty:
        print(f"Warning: Bin {bin_id} not found in data")
        return
    
    # Extract measurements for this bin
    bin_data = pd.DataFrame({
        'sample_id': data.columns[1:],
        'measurement': bin_row.iloc[0, 1:].values
    })
    
    # Merge with metadata
    bin_data = pd.merge(bin_data, metadata, on='sample_id')
    
    # Convert factors to strings (in case they're numeric)
    bin_data[factor1_name] = bin_data[factor1_name].astype(str)
    bin_data[factor2_name] = bin_data[factor2_name].astype(str)
    
    # Create a figure
    plt.figure(figsize=(10, 6))
    
    # Use seaborn for better visualization
    sns.pointplot(data=bin_data, x=factor1_name, y='measurement', hue=factor2_name, 
                 dodge=True, errorbar=('se', 1), capsize=0.2)
    
    # Add title and subtitle with p-values
    plt.title(f"Bin: {bin_id}")
    plt.suptitle(
        f"{factor1_name} p = {stats_row['factor1_pvalue']:.3e}, "
        f"{factor2_name} p = {stats_row['factor2_pvalue']:.3e}, "
        f"Interaction p = {stats_row['interaction_pvalue']:.3e}",
        y=0.92, fontsize=9
    )
    
    plt.ylabel('Oxidation Level')
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(f"{output_file_prefix}.png", dpi=300)
    plt.savefig(f"{output_file_prefix}.pdf")
    plt.close()



In [ ]:
def main():
    """Main function to run the entire analysis pipeline"""
    import argparse
    
    parser = argparse.ArgumentParser(description='Perform two-way ANOVA on thousands of genomic bins')
    parser.add_argument('--data', required=True, help='CSV file with bin data')
    parser.add_argument('--metadata', required=True, help='CSV file with sample metadata')
    parser.add_argument('--factor1', required=True, help='Name of the first factor in metadata')
    parser.add_argument('--factor2', required=True, help='Name of the second factor in metadata')
    parser.add_argument('--output', required=True, help='Prefix for output files')
    parser.add_argument('--cores', type=int, default=None, help='Number of CPU cores to use')
    parser.add_argument('--max-bins', type=int, default=10, 
                       help='Maximum number of top bins to visualize for each factor')
    
    args = parser.parse_args()
    
    # Run the analysis
    results_df = analyze_genomic_bins(
        args.data, args.metadata, args.factor1, args.factor2, args.output, args.cores
    )
    
    # Create visualizations
    visualize_anova_results(results_df, args.factor1, args.factor2, args.output)
    
    # Analyze significant bins
    analyze_significant_bins(
        args.data, args.metadata, results_df, args.factor1, args.factor2, args.output, args.max_bins
    )

if __name__ == '__main__':
    main()

# Example usage:
# python genome_anova.py \
#   --data genomic_bins_measurements.csv \
#   --metadata sample_metadata.csv \
#   --factor1 treatment \
#   --factor2 timepoint \
#   --output oxidation_analysis \
#   --cores 4 \
#   --max-bins 10